In [29]:
import pandas as pd
import numpy as np

from deploy.data_prep_functions import *

In [2]:
base_player = pd.read_csv("bases_nba_stats/aux_files/base_nba_per_player_16_17.csv")

In [5]:
descricao = pd.read_excel("old_files/descricao_colunas.xlsx")

In [10]:
colunas_defensive = [x.strip() for x in descricao[descricao.book == "defensive"].column_name]

In [22]:
base_player["fl_home"] = np.where(base_player["GAME"].str[6:9] == base_player["TEAM_ABBREVIATION"], 1, 0)

In [57]:
defensive_base_player = base_player[["GAME", "GAME_DATE", "TEAM_ABBREVIATION", "fl_home", "PTS"] + colunas_defensive].groupby(["GAME", "TEAM_ABBREVIATION", "GAME_DATE"]).sum().reset_index()
defensive_base_player = defensive_base_player.sort_values(["GAME_DATE", "GAME"], ascending=True)

In [59]:
home_games = defensive_base_player[defensive_base_player["fl_home"] != 0].set_index("GAME")
away_games = defensive_base_player[defensive_base_player["fl_home"] == 0].set_index("GAME")

In [60]:
all_games = home_games.join(away_games, how="inner", lsuffix="_home", rsuffix="_away")
all_games.drop(["GAME_ID_away", "GAME_DATE_away", "GAME_PLACE_away",
            "MIN_home", "MIN_away", 'PTS_hustle_home', 'PTS_hustle_away',
            "fl_home_away", "fl_home_home"], axis=1, inplace=True, errors="ignore")

all_games["DATE"] = [datetime.strptime(str(x), '%Y-%m-%d') for x in all_games.GAME_DATE_home]
all_games['fl_home_win'] = np.where(all_games['PTS_home'] > all_games['PTS_away'], 1, 0)
all_games = all_games.sort_values('DATE')

all_games = all_games.rename(columns={'TEAM_ABBREVIATION_home': "team_home",
                                 'TEAM_ABBREVIATION_away': "team_away"})

all_games["team_home_game_num"] = all_games.groupby(['team_home']).cumcount() + 1
all_games["team_away_game_num"] = all_games.groupby(['team_away']).cumcount() + 1

In [61]:
all_games

,team_home,GAME_DATE_home,PTS_home,POSS_home,PLAYER_PTS_home,TEAM_PTS_home,AST_defensive_home,TOV_home,DREB_defensive_home,STL_defensive_home,...,CFGM_defensive_away,CFGA_defensive_away,CFG_PCT_defensive_away,CFG3M_away,CFG3A_away,CFG3_PCT_away,DATE,fl_home_win,team_home_game_num,team_away_game_num
GAME,,,,,,,,,,,,,,,,,,,,,
NYK @ CLE 2016-10-25,CLE,2016-10-25,117.0,481,86,430,17,17,40,12,...,17,39,4.711,2,3,1.500,2016-10-25,1,1,1
SAS @ GSW 2016-10-25,GSW,2016-10-25,100.0,492,127,650,25,12,27,11,...,21,38,5.967,1,6,0.333,2016-10-25,0,1,1
UTA @ POR 2016-10-25,POR,2016-10-25,113.0,450,102,526,18,10,29,5,...,19,43,2.911,4,5,2.667,2016-10-25,1,1,1
SAC @ PHX 2016-10-26,PHX,2016-10-26,94.0,451,109,586,23,12,31,6,...,18,38,5.000,1,4,1.000,2016-10-26,0,1,1
OKC @ PHI 2016-10-26,PHI,2016-10-26,97.0,496,105,525,19,10,38,5,...,14,33,5.533,0,0,0.000,2016-10-26,0,1,1
MIA @ ORL 2016-10-26,ORL,2016-10-26,96.0,473,108,540,25,10,30,5,...,16,48,3.424,1,2,1.000,2016-10-26,0,1,1
HOU @ LAL 2016-10-26,LAL,2016-10-26,120.0,492,111,559,29,13,28,10,...,23,38,4.982,1,5,0.500,2016-10-26,1,1,1
MIN @ MEM 2016-10-26,MEM,2016-10-26,102.0,444,92,497,20,8,34,6,...,13,31,2.492,0,1,0.000,2016-10-26,1,1,1
DEN @ NOP 2016-10-26,NOP,2016-10-26,102.0,514,100,512,17,20,28,16,...,21,45,3.843,0,2,0.000,2016-10-26,0,1,1
